In [3]:
import math

In [1]:
import sys
print(sys.executable)


/Users/hamzamunawar/opt/anaconda3/bin/python


In [2]:
!which pip


/Users/hamzamunawar/opt/anaconda3/bin/pip


In [5]:
#!/usr/bin/env python  
'''
Reference code for data loading. 
Author: Chongyang Bai
For more details, refer to the paper:
C.Bai, S. Kumar, J. Leskovec, M. Metzger, J.F. Nunamaker, V.S. Subrahmanian,
Predicting Visual Focus of Attention in Multi-person Discussion Videos,
International Joint Conference on Artificial Intelligence (IJCAI), 2019.
'''

import numpy as np
import pandas as pd

In [6]:
def loadGame(game, N):
	# N is the number of players in the game
	# load csv data
	# unweighted and weighted networks can be loaded exactly the same way
	# below shows the loader for weighted networks
	df_network = pd.read_csv(f'{src}/network{game}_weighted.csv', index_col = 0)

	# T is number of timestamps (10 frames)
	T = len(df_network)
	# load VFOA network to T x N x (N+1) array
	# vfoa[t, n, i] is the probability of player n+1 looking at object i at time t
	# i: 0 - laptop, 1 - player 1, 2 - player 2, ..., N - player N
	vfoa = np.reshape(df_network.values, (T,N,N+1))

	# print information
	print(f'network id:{game}\t length(x 1/3 second): {T}\t num of players: {N}')
	return vfoa

src = './network' # root dir of data
meta = pd.read_csv('network_list.csv')

for _, row in meta.iterrows():
	loadGame(row['NETWORK'], row['NUMBER_OF_PARTICIPANTS'])

network id:0	 length(x 1/3 second): 7323	 num of players: 7
network id:1	 length(x 1/3 second): 5328	 num of players: 8
network id:2	 length(x 1/3 second): 7533	 num of players: 7
network id:3	 length(x 1/3 second): 6264	 num of players: 7
network id:4	 length(x 1/3 second): 7323	 num of players: 8
network id:5	 length(x 1/3 second): 9651	 num of players: 8
network id:6	 length(x 1/3 second): 7260	 num of players: 7
network id:7	 length(x 1/3 second): 7245	 num of players: 6
network id:8	 length(x 1/3 second): 6243	 num of players: 7
network id:9	 length(x 1/3 second): 8340	 num of players: 8
network id:10	 length(x 1/3 second): 6681	 num of players: 7
network id:11	 length(x 1/3 second): 5415	 num of players: 7
network id:12	 length(x 1/3 second): 5571	 num of players: 8
network id:13	 length(x 1/3 second): 7173	 num of players: 7
network id:14	 length(x 1/3 second): 9429	 num of players: 8
network id:15	 length(x 1/3 second): 5466	 num of players: 6
network id:16	 length(x 1/3 second

In [7]:
meta

,NETWORK,NUMBER_OF_PARTICIPANTS
0,0,7
1,1,8
2,2,7
3,3,7
4,4,8
...,...,...
57,57,8
58,58,8
59,59,7
60,60,7


In [9]:
net0 = loadGame(0, 7)

network id:0	 length(x 1/3 second): 7323	 num of players: 7


In [ ]:
def loadGame(game, N):
	# N is the number of players in the game
	# load csv data
	# unweighted and weighted networks can be loaded exactly the same way
	# below shows the loader for weighted networks
	df_network = pd.read_csv(f'{src}/network{game}_weighted.csv', index_col = 0)

	# T is number of timestamps (10 frames)
	T = len(df_network)
	# load VFOA network to T x N x (N+1) array
	# vfoa[t, n, i] is the probability of player n+1 looking at object i at time t
	# i: 0 - laptop, 1 - player 1, 2 - player 2, ..., N - player N
	vfoa = np.reshape(df_network.values, (T,N,N+1))

	# print information
	print(f'network id:{game}\t length(x 1/3 second): {T}\t num of players: {N}')
	return vfoa

src = './network' # root dir of data
meta = pd.read_csv('network_list.csv')

for _, row in meta.iterrows():
	loadGame(row['NETWORK'], row['NUMBER_OF_PARTICIPANTS'])

In [12]:
def convert_to_edge_list(vfoa, N):
    edge_lists = []
    T = vfoa.shape[0]
    
    for t in range(T):
        time_edges = []
        for n in range(N):
            for i in range(N + 1):
                if i == 0:
                    target = 'LAPTOP'
                else:
                    target = f'P{i}'
                weight = vfoa[t, n, i]
                if weight > 0:
                    source = f'P{n+1}'
                    time_edges.append((source, target, weight))
        
        # Add timestamp to each edge
        timestamped_edges = [(t * (1/3), edge[0], edge[1], edge[2]) for edge in time_edges]
        edge_lists.extend(timestamped_edges)
    
    return edge_lists

# Convert net0 to edge list
edge_list_net0 = convert_to_edge_list(net0, 7)

# Create a DataFrame for easier handling in R
df_edge_list_net0 = pd.DataFrame(edge_list_net0, columns=['timestamp', 'from', 'to', 'weight'])

# # Save to CSV for import in R
# df_edge_list_net0.to_csv('edge_list_net0.csv', index=False)

In [16]:
df_edge_list_net0.to_csv('edge_list_net0.csv', index=False)